# Multi-Agent Resume Tailoring System
### Capstone Project - Google & Kaggle AI Agent Intensive Course

## Project Overview

This project implements a multi-agent system using Google's Agent Development Kit (ADK). The pipeline automates resume tailoring by combining:

1. **Resume Intake Agent** – extracts and standardizes resume content  
2. **Job Research Agent** – performs live search to identify real job requirements  
3. **Resume Rewrite Agent** – merges resume + job insights into a tailored final resume  
4. **Polishing Agent** – refines clarity, style, and conciseness

## Architecture Diagram

## Agent Definitions

## Pipeline Design

## 

In [2]:
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"API Authentication Error, please make sure you have setup your .env with the correct GOOGLE_API_KEY: {e}")

✅ Gemini API key setup complete.


In [29]:
from google.adk.agents import Agent, LlmAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.models import Gemini
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
import google.generativeai as genai

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

🧹 Cleaned up logger.log
✅ Logging configured


In [9]:
# Automatic retries to contact the Gemini API if it fails
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agent Instructions

Each agent recieves a different set of instructions tailored to its role. This seperation is key for improving perforance, security, and manageability as we learned in the course.

In [1]:
INTAKE_INSTRUCTION = """
    Extract structured resume info (education, skills, projects, experience).
    Return bullet points without rewriting or inventing details.
"""

JOB_RESEARCH_INSTRUCTION = """
    Perform a Google search for the given job title/company.
    Summarize required skills, preferred qualifications, and responsibilities.
"""

REWRITE_INSTRUCTION = """
    Combine the parsed resume + job research.
    Rewrite the resume using strong, tailored bullet points.
    Do NOT invent false experience.
"""

POLISH_INSTRUCTION = """
    Clean, format, and simplify the improved resume.
    Ensure conciseness, clarity, and consistent style.
"""

print("✅ Agents instructions created")

✅ Agents instructions created


## Defining Agents and Runners

In [11]:
resume_intake_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeIntakeAgent",
    instruction=INTAKE_INSTRUCTION,
    tools=[]
)

job_research_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="JobResearchAgent",
    instruction=JOB_RESEARCH_INSTRUCTION,
    tools=[google_search]
)

resume_rewrite_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="ResumeRewriteAgent",
    instruction=REWRITE_INSTRUCTION,
    tools=[]
)

polish_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="PolishAgent",
    instruction=POLISH_INSTRUCTION,
    tools=[]
)

# Defining Runners
intake_runner = InMemoryRunner(resume_intake_agent)
job_runner = InMemoryRunner(job_research_agent)
rewrite_runner = InMemoryRunner(resume_rewrite_agent)
polish_runner = InMemoryRunner(polish_agent)

print("✅ Agents + Runners created")

✅ Agents + Runners created


## Multi-Agent Pipeline

In [31]:
import contextlib
import io

async def silent_run_debug(runner, text):
    buffer = io.StringIO()
    with contextlib.redirect_stdout(buffer):
        events = await runner.run_debug(text)

    last_text = None

    for event in events:
        if hasattr(event, "content") and event.content:
            parts = event.content.parts
            if parts and hasattr(parts[0], "text"):
                last_text = parts[0].text

    return last_text

async def agent_pipeline(resume_text, job_query):
    intake_output = await silent_run_debug(intake_runner, resume_text)
    job_output = await silent_run_debug(job_runner, job_query)

    resume_prompt = f"""
    Parsed resume:
    {intake_output}

    Job research:
    {job_output}

    Rewrite the resume using BOTH.
    """
    rewrite_output = await silent_run_debug(rewrite_runner, resume_prompt)
    polish_output = await silent_run_debug(polish_runner, rewrite_output)

    return rewrite_output

print("✅ Pipeline + helper functions created")

✅ Pipeline + helper functions created


### Fake Resume Used as Example

In [27]:
FAKE_RESUME = """
ALEX CARTER
alex.carter@example.com | (555) 291-8743 | San Diego, CA
github.com/alexcarter-dev | linkedin.com/in/alexcarter

EDUCATION
University of California, San Diego
B.S. in Computer Science, Expected June 2026
Relevant Coursework: Data Structures & Algorithms, Machine Learning, Operating Systems, Database Systems, Software Engineering

TECHNICAL SKILLS
Languages: Python, Java, C++, SQL, JavaScript
Tools & Frameworks: PyTorch, TensorFlow, React, Node.js, Flask, Docker, Git
Concepts: Machine Learning, REST APIs, Distributed Systems, Data Pipelines

PROJECTS
SmartTransit (Python, ML)
• Built a real-time bus arrival prediction model using LSTM networks, improving prediction accuracy by 18%.
• Designed a pipeline to clean, merge, and process 2M+ GPS datapoints from a public transit dataset.
• Containerised the entire system using Docker to support scalable deployment.

PixelForge (React, Node.js)
• Developed a full-stack collaborative drawing platform with real-time canvas syncing over WebSockets.
• Implemented role-based access control and persistent project saving with MongoDB.
• Improved frontend load performance by 30% via code splitting and caching.

GPU Weather Simulator (C++, CUDA)
• Accelerated fluid-dynamics-based weather simulation by 12× using CUDA kernels.
• Wrote custom memory-optimised kernels for particle advection and temperature diffusion.

WORK EXPERIENCE
Software Engineering Intern — Horizon Analytics (June 2024 – September 2024)
• Implemented data processing modules for a telemetry monitoring platform ingesting 30K+ events/sec.
• Wrote Python ETL jobs responsible for 15% overall speed improvement.
• Built automated integration tests using pytest and GitHub Actions.

LEADERSHIP & ACTIVITIES
Computer Science Society — Project Team Lead
• Led a 6-student team developing a campus navigation app with indoor routing.
• Managed weekly standups, roadmap planning, and code reviews.

Hackathons
• Winner (1st Place), SD Hacks 2024 — built a wildfire-risk prediction dashboard using satellite imagery.

ADDITIONAL
• Strong interests in ML engineering, distributed systems, and large-scale backend systems.
• Fluent in English and conversational in Spanish.
"""

### Running the Process

In [32]:
#resume_text = input("Paste your resume: ")
resume_text = FAKE_RESUME
job_query = input("Enter a job (e.g., 'Tesla SWE Intern 2026'): ")

result = await agent_pipeline(resume_text, job_query)

print("\n\n================== POLISHED RESUME ==================\n\n", result)

Enter a job (e.g., 'Tesla SWE Intern 2026'):  Tesla Data Science Intern 2026


================== POLISHED RESUME ==================

 ## Education

**University of California, San Diego** — B.S. in Computer Science, Expected June 2026
*   Relevant Coursework: Data Structures & Algorithms, Machine Learning, Operating Systems, Database Systems, Software Engineering

## Technical Skills

*   **Languages:** Python, Java, C++, SQL, JavaScript
*   **Frameworks & Libraries:** PyTorch, TensorFlow, React, Node.js, Flask, Pandas, NumPy, pytest
*   **Tools & Technologies:** Docker, Git, GitHub Actions, MongoDB, CUDA
*   **Concepts:** Machine Learning, Deep Learning, Statistical Learning, REST APIs, Distributed Systems, Data Pipelines, Data Analysis, Data Visualization, WebSockets, MLOps

## Projects

*   **SmartTransit: Real-time Bus Arrival Prediction Model**
    *   Developed and deployed a real-time bus arrival prediction model using LSTM networks in Python, achieving an 18% improvement in prediction accuracy.
    *   Engineered a comprehensive data pipeline to clean, m